# Connected Component Analysis

In [ ]:
import this

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color
import skimage.filters
import skimage.measure
%matplotlib widget

In [ ]:
%load_ext watermark
%watermark -v -p numpy,matplotlib,skimage
%matplotlib

In [ ]:
image = skimage.io.imread("fig/junk.jpg")

fig, ax = plt.subplots()
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
gray_image = skimage.color.rgb2gray(image)

sigma = 2.0
blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)

t = 0.9
binary_mask = blurred_image < t

fig, ax = plt.subplots()
plt.imshow(binary_mask, cmap='gray')
plt.axis('off')
plt.show()

## Connected Component Analysis

In [ ]:
connectivity = 2
labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)

fig, ax = plt.subplots()
plt.imshow(labeled_image)
plt.axis('off')
plt.show()

In [ ]:
colored_label_image = skimage.color.label2rgb(labeled_image, bg_label=0)

fig, ax = plt.subplots()
plt.imshow(colored_label_image)
plt.axis('off')
#plt.savefig("fig/09-labeled-objects.png",dpi=150)
plt.show()

### How many objects are in that image

In [ ]:
def connected_components(filename, sigma=1.0, t=0.5, connectivity=2):
    image = skimage.io.imread(filename)
    gray_image = skimage.color.rgb2gray(image)
    blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < t
    labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)
    return labeled_image, count

labeled_image, count = connected_components("fig/junk.jpg", sigma=2.0, t=0.9, connectivity=2)

fig, ax = plt.subplots()
plt.imshow(labeled_image)
plt.axis('off')
plt.show()

In [ ]:
print("Found", count, "objects in the image.")

In [ ]:
num_objects = np.max(labeled_image)
print("Found", num_objects, "objects in the image.")

In [ ]:
fig, ax = plt.subplots()
plt.imshow(colored_label_image)
plt.axis('off')
plt.xlim(0,200)
plt.ylim(1700,1300)
plt.show()

## Morphometrics - Describe object features with numbers

In [ ]:
# compute object features and extrac object areas
object_features = skimage.measure.regionprops(labeled_image)
object_areas = [objf["area"] for objf in object_features]
object_areas

### Plot a histogram of the object area distribution

In [ ]:
fig, ax = plt.subplots()
plt.hist(object_areas)
plt.xlabel("Area (pixels)", size=14)
plt.ylabel("Number of objects", size=14)
plt.tight_layout()
#plt.savefig("fig/08-areas-histogram.png")
plt.show()

### Filter objects by area

In [ ]:
min_area = 200
large_objects = [ area for area in object_areas if area > min_area ]

fig, ax = plt.subplots()
plt.hist(large_objects)
plt.show()

In [ ]:
large_objects = []
for objf in object_features:
    if objf["area"] > min_area:
        large_objects.append(objf["label"])
print("Found", len(large_objects), "objects!")

In [ ]:
object_areas = np.array([objf["area"] for objf in object_features])
object_labels = np.array([objf["label"] for objf in object_features])
large_objects = object_labels[object_areas > min_area]
print("Found", len(large_objects), "objects!")

In [ ]:
n = np.count_nonzero(object_areas > min_area)
print("Found", n, "objects!")

### Remove small objects

In [ ]:
for object_id, objf in enumerate(object_features, start=1):
    if objf["area"] < min_area:
        labeled_image[labeled_image == objf["label"]] = 0

In [ ]:
object_areas = np.array([objf["area"] for objf in object_features])
object_labels = np.array([objf["label"] for objf in object_features])
small_objects = object_labels[object_areas < min_area]
labeled_image[np.isin(labeled_image,small_objects)] = 0

In [ ]:
object_mask = skimage.morphology.remove_small_objects(binary_mask,min_area)
labeled_image, n = skimage.measure.label(object_mask, connectivity=connectivity, return_num=True)

In [ ]:
def enhanced_connected_components(filename, sigma=1.0, t=0.5, connectivity=2, min_area=0):
    image = skimage.io.imread(filename)
    gray_image = skimage.color.rgb2gray(image)
    blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < t
    labeled_image, count = skimage.measure.label(binary_mask, connectivity=connectivity, return_num=True)
    object_features = skimage.measure.regionprops(labeled_image)
    object_areas = np.array([objf["area"] for objf in object_features])
    object_labels = np.array([objf["label"] for objf in object_features])
    labeled_image[np.isin(labeled_image,object_labels[object_areas < min_area])] = 0
    return labeled_image, np.count_nonzero(object_areas > min_area)

labeled_image, count = enhanced_connected_components("fig/junk.jpg", sigma=2.0, t=0.9, connectivity=2, min_area=min_area)
colored_label_image = skimage.color.label2rgb(labeled_image, bg_label=0)

fig, ax = plt.subplots()
plt.imshow(colored_label_image)
plt.axis('off')
#plt.savefig("fig/08-filtered-objects.png",dpi=150)
plt.show()

print("Found", count, "objects in the image.")

In [ ]:
def enhanced_connected_components(filename, sigma=1.0, t=0.5, connectivity=2, min_area=0):
    image = skimage.io.imread(filename)
    gray_image = skimage.color.rgb2gray(image)
    blurred_image = skimage.filters.gaussian(gray_image, sigma=sigma)
    binary_mask = blurred_image < t
    object_mask = skimage.morphology.remove_small_objects(binary_mask,min_area)
    labeled_image, count = skimage.measure.label(object_mask, connectivity=connectivity, return_num=True)
    return labeled_image, count

labeled_image, count = enhanced_connected_components("fig/junk.jpg", sigma=2.0, t=0.9, connectivity=2, min_area=min_area)
colored_label_image = skimage.color.label2rgb(labeled_image, bg_label=0)

fig, ax = plt.subplots()
plt.imshow(colored_label_image)
plt.axis('off')
#plt.savefig("fig/08-filtered-objects.png",dpi=150)
plt.show()

print("Found", count, "objects in the image.")

In [ ]:
object_areas = np.array([objf["area"] for objf in skimage.measure.regionprops(labeled_image)])
colored_area_image = np.insert(0,1,object_areas)[labeled_image]

fig, ax = plt.subplots()
im = plt.imshow(colored_area_image)
cbar = fig.colorbar(im, ax=ax, shrink=0.85)
cbar.ax.set_title('Area')
plt.axis('off')
#plt.savefig("fig/08-objects-colored-by-area.png",dpi=150)
plt.show()